# Tanzania Water Wells Project

Morgan Nash

August 2025

# Overview

## Business Understanding

clean drinking water, look deeper to see looking at different features like pump creation or water to predict pump is functional or needs repair



In [1]:
#predict which water pumps are faulty to promote access to clean, potable water across Tanzania
### copied but helpful for further perspective in going into the proj: 
#global climate change->water scarcity->increasing problem in Tanzania, either intense and destructive rainfall or long dry spells.


## Data Understanding

The data for this project comes from Taarifa, who compiled data from the Tanzania Ministry of Water, and was accessed through DrivenData.org. The data contains 54,900 records of water wells, each with 41 features, although many are duplicates. Our target is **status_group** which contains labeling of whether a pump is functional, functional needs repair, or non functional. After cleaning the data, we'll be using the features to build a classification model to predict the status of water wells.


Training Labels Dataset:
* id: Unique identifier for each water pump
* status_group: contains labels whether a pump is functional, functional needs repair, or non functional

* status_group will be our target

Training Values Dataset:
* id: Unique identifier for each water pump
* amount_tsh: Total static head (amount of water available to pump)
* date_recorded: Date the pump data was recorded
* funder: Person or org funded the pump installation
* gps_height: Altitude of the pump location
* installer: Person or org that installed the pump
* longitude: GPS longitude coordinate of the pump location
* latitude: GPS latitude coordinate of the pump location
* wpt_name: Name of the waterpoint (if available)
* num_private: Number of private plots reserved for the waterpoint
* basin: Geographic basin of the pump location
* subvillage: Geographic location within the village
* region: Geographic location
* region_code: Coded- geographic region
* district_code: Coded- administrative district
* lga: Geographic location (Local Government Area)
* ward: Geographic location (Administrative division)
* population: Population served by the well
* public_meeting: T/F Indicator of whether there was a public meeting about the well
* recorded_by: Group entering this row of data
* scheme_management: Who operates the waterpoint
* scheme_name: Who operates the waterpoint
* permit: Indicator of whether the waterpoint is permitted
* construction_year: Year the pump was installed
* extraction_type: The kind of extraction the waterpoint uses
* extraction_type_group: The kind of extraction the waterpoint uses
* extraction_type_class: The kind of extraction the waterpoint uses
* management: Type of management for the pump
* management_group: Grouped management type
* payment: Payment type for water service
* payment_type: Type of payment
* water_quality: Quality of water provided by the pump
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint

# Exploratory Data Analysis

## Data Preparation & Cleaning

In [2]:
#import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

In [92]:
#load the data
val = pd.read_csv('data/Training_set_values.csv')
val.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


At first glance, it looks like there's many categorical columns that are similar or duplicates suggesting multicolinearity, as well as some categorical columns that will need to be OneHotEncoded!

In [93]:
#load the data
labels = pd.read_csv('data/Training_set_labels.csv')
labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [94]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [95]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [96]:
labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [97]:
# Check target class (im)balance with percentages: Pretty significant imbalance!!
labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

### Make the target binary:

In [98]:
# make problem binary: change the status_group labels 'non functional' and 'functinoal needs repair' into 'needs repair'
labels['status_group'] = labels['status_group'].map({'non functional':'needs repair',
                                                       'functional needs repair':'needs repair',
                                                       'functional':'functional'})

In [99]:
labels['status_group'].value_counts()

functional      32259
needs repair    27141
Name: status_group, dtype: int64

In [100]:
#although not all, this fixes a good amount of class imbalance
labels['status_group'].value_counts(normalize=True)

functional      0.543081
needs repair    0.456919
Name: status_group, dtype: float64

### Combine Values & Labels into One Dataframe

In [101]:
# Merge the two dataframes using shared 'id' column
df = pd.merge(val, labels, how = 'left', on='id')
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,needs repair
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [102]:
#Check for nulls
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [103]:
#scheme_name has missing values for over half of the records
#funder and installer exact same amount nulls

In [104]:
#columns containing nulls: funder, installer, subvillage, public_meeting, scheme_management, scheme_name, permit

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

### Cleaning Categorical Columns:

Decide which columns to drop, which that have missing values, and which that we will One Hot Encode after splitting:

In [106]:
#Look further at categorical columns
#compare columns that seem to have overlap and decide which to keep/which to get rid of (ensuring no multicolinearity)
categorical = [var for var in df.columns if df[var].dtype=='O']
categorical

['date_recorded',
 'funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'lga',
 'ward',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group',
 'status_group']

In [138]:
#same value for every record, will drop
df['recorded_by'].value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [109]:
df['basin'].value_counts()

Lake Victoria              10248
Pangani                     8940
Rufiji                      7976
Internal                    7785
Lake Tanganyika             6432
Wami / Ruvu                 5987
Lake Nyasa                  5085
Ruvuma / Southern Coast     4493
Lake Rukwa                  2454
Name: basin, dtype: int64

In [110]:
#management group is slightly broader categories of the management column, I'll keep management column
df[['management_group', 'management']].value_counts()

management_group  management      
user-group        vwc                 40507
                  wug                  6515
                  water board          2933
                  wua                  2535
commercial        private operator     1971
parastatal        parastatal           1768
commercial        water authority       904
other             other                 844
commercial        company               685
unknown           unknown               561
other             other - school         99
commercial        trust                  78
dtype: int64

In [111]:
#waterpoint_type_group and waterpoint_type columns contain the same information:
df[['waterpoint_type_group', 'waterpoint_type']].value_counts()

waterpoint_type_group  waterpoint_type            
communal standpipe     communal standpipe             28522
hand pump              hand pump                      17488
other                  other                           6380
communal standpipe     communal standpipe multiple     6103
improved spring        improved spring                  784
cattle trough          cattle trough                    116
dam                    dam                                7
dtype: int64

In [112]:
#funder and installer columns have the same amount of missing values, and both have messy strings that will require cleaning.
df[['funder', 'installer']].value_counts()

funder                          installer           
Government Of Tanzania          DWE                     4254
                                Government              1607
Hesawa                          DWE                     1296
Danida                          DANIDA                  1046
Rwssp                           DWE                      914
                                                        ... 
Ola                             OLS                        1
Okutu Village Community         Gerald Mila                1
Oikos E.Africa/ European Union  Oikos E .Africa            1
Oda                             VILLAGE COUNCIL .ODA       1
Msikiti Masji                   ms                         1
Length: 3697, dtype: int64

In [113]:
len(df['installer'].value_counts())

2145

In [114]:
#The following code cell is copied from Leonard Gachimu. It cleans up the installer column.
#https://github.com/leogachimu/Tanzania_Water_Wells_Classification/blob/main/student.ipynb

### Replace close variations and misspellings in the installer column

df['installer'] = df['installer'].replace(to_replace = ('Central government', 'Tanzania Government',
                                          'Cental Government','Tanzania government','Cebtral Government', 
                                          'Centra Government', 'central government', 'CENTRAL GOVERNMENT', 
                                          'TANZANIA GOVERNMENT', 'TANZANIAN GOVERNMENT', 'Central govt', 
                                          'Centr', 'Centra govt', 'Tanzanian Government', 'Tanzania', 
                                          'Tanz', 'Tanza', 'GOVERNMENT', 
                                          'GOVER', 'GOVERNME', 'GOVERM', 'GOVERN', 'Gover', 'Gove', 
                                          'Governme', 'Governmen', 'Got', 'Serikali', 'Serikari', 'Government',
                                          'Central Government'), 
                                          value = 'Central Government')

df['installer'] = df['installer'].replace(to_replace = ('IDARA', 'Idara ya maji', 'MINISTRY OF WATER',
                                          'Ministry of water', 'Ministry of water engineer', 'MINISTRYOF WATER', 
                                          'MWE &', 'MWE', 'Wizara ya maji', 'WIZARA', 'wizara ya maji',
                                          'Ministry of Water'), 
                                          value ='Ministry of Water')

df['installer'] = df['installer'].replace(to_replace = ('District COUNCIL', 'DISTRICT COUNCIL',
                                          'Counc','District council','District Counci', 
                                          'Council', 'COUN', 'Distri', 'Halmashauri ya wilaya',
                                          'Halmashauri wilaya', 'District Council'), 
                                          value = 'District  Council')

df['installer'] = df['installer'].replace(to_replace = ('District water depar', 'District Water Department', 
                                          'District water department', 'Distric Water Department'),
                                          value = 'District Water Department')

df['installer'] = df['installer'].replace(to_replace = ('villigers', 'villager', 'villagers', 'Villa', 'Village',
                                          'Villi', 'Village Council', 'Village Counil', 'Villages', 'Vill', 
                                          'Village community', 'Villaers', 'Village Community', 'Villag',
                                          'Villege Council', 'Village council', 'Villege Council', 'Villagerd', 
                                          'Villager', 'VILLAGER', 'Villagers',  'Villagerd', 'Village Technician', 
                                          'Village water attendant', 'Village Office', 'VILLAGE COUNCIL',
                                          'VILLAGE COUNCIL .ODA', 'VILLAGE COUNCIL Orpha', 'Village community members', 
                                          'VILLAG', 'VILLAGE', 'Village Government', 'Village government', 
                                          'Village Govt', 'Village govt', 'VILLAGERS', 'VILLAGE WATER COMMISSION',
                                          'Village water committee', 'Commu', 'Communit', 'commu', 'COMMU', 'COMMUNITY', 
                                           'Comunity', 'Communit', 'Kijiji', 'Serikali ya kijiji', 'Community'), 
                                          value ='Community')

df['installer'] = df['installer'].replace(to_replace = ('FinW', 'Fini water', 'FINI WATER', 'FIN WATER',
                                          'Finwater', 'FINN WATER', 'FinW', 'FW', 'FinWater', 'FiNI WATER', 
                                          'FinWate', 'FINLAND', 'Fin Water', 'Finland Government'), 
                                          value ='Finnish Government')

df['installer'] = df['installer'].replace(to_replace = ('RC CHURCH', 'RC Churc', 'RC', 'RC Ch', 'RC C', 'RC CH',
                                          'RC church', 'RC CATHORIC', 'Roman Church', 'Roman Catholic',
                                          'Roman catholic', 'Roman Ca', 'Roman', 'Romam', 'Roma', 
                                          'ROMAN CATHOLIC', 'Kanisa', 'Kanisa katoliki'), 
                                          value ='Roman Catholic Church')

df['installer'] = df['installer'].replace(to_replace = ('Dmdd', 'DMDD'), value ='DMDD') 

df['installer'] = df['installer'].replace(to_replace = ('TASA', 'Tasaf', 'TASAF 1', 'TASAF/', 'TASF',
                                          'TASSAF', 'TASAF'), value ='TASAF') 

df['installer'] = df['installer'].replace(to_replace = ('RW', 'RWE'), value ='RWE')

df['installer'] = df['installer'].replace(to_replace = ('SEMA CO LTD', 'SEMA Consultant', 'SEMA'), value ='SEMA')

df['installer'] = df['installer'].replace(to_replace = ('DW E', 'DW#', 'DW$', 'DWE&', 'DWE/', 'DWE}', 
                                         'DWEB', 'DWE', 'DW'), value ='DWE')

df['installer'] = df['installer'].replace(to_replace = ('No', 'NORA', 'Norad', 'NORAD/', 'NORAD'), 
                                          value ='NORAD') 

df['installer'] = df['installer'].replace(to_replace = ('Ox', 'OXFARM', 'OXFAM'), value ='OXFAM') 

df['installer'] = df['installer'].replace(to_replace = ('PRIV', 'Priva', 'Privat', 'private', 'Private company',
                                          'Private individuals', 'PRIVATE INSTITUTIONS', 'Private owned',
                                          'Private person', 'Private Technician', 'Private'), 
                                          value ='Private') 

df['installer'] = df['installer'].replace(to_replace = ('Ch', 'CH', 'Chiko', 'CHINA', 'China',
                                            'China Goverment'), value ='Chinese Goverment')

df['installer'] = df['installer'].replace(to_replace = ('Unisef','Unicef', 'UNICEF'), value ='UNICEF')
                                          
df['installer'] = df['installer'].replace(to_replace = ('Wedeco','WEDEKO', 'WEDECO'), value ='WEDECO')

df['installer'] = df['installer'].replace(to_replace = ('Wo','WB', 'Word Bank', 'Word bank', 'WBK',
                                          'WORDL BANK', 'World', 'world', 'WORLD BANK', 'World bank',
                                          'world banks', 'World banks', 'WOULD BANK', 'World Bank'), 
                                          value ='World Bank')
                                          
df['installer'] = df['installer'].replace(to_replace = ('Lga', 'LGA'), value ='LGA')

df['installer'] = df['installer'].replace(to_replace = ('World Division', 'World Visiin', 
                                         'World vision', 'WORLD VISION', 'world vision', 'World Vission', 
                                          'World Vision'), 
                                          value ='World Vision')

df['installer'] = df['installer'].replace(to_replace = ('Local', 'Local technician', 'Local  technician',
                                         'local  technician', 'LOCAL CONTRACT', 'local fundi', 
                                         'Local l technician', 'Local te', 'Local technical', 'Local technical tec',
                                         'local technical tec', 'local technician', 'Local technitian',
                                         'local technitian', 'Locall technician', 'Localtechnician',
                                         'Local Contractor'), 
                                          value ='Local Contractor')
                                          
df['installer'] = df['installer'].replace(to_replace = ('DANID', 'DANNY', 'DANNIDA', 'DANIDS', 
                                         'DANIDA CO', 'DANID', 'Danid', 'DANIAD', 'Danda', 'DA',
                                         'DENISH', 'DANIDA'), 
                                          value ='DANIDA')

df['installer'] = df['installer'].replace(to_replace =('Adrs', 'Adra', 'ADRA'), value ='ADRA')
                                          
df['installer'] = df['installer'].replace(to_replace = ('Hesawa', 'hesawa', 'HESAW', 'hesaw',
                                          'HESAWQ', 'HESAWS', 'HESAWZ', 'hesawz', 'hesewa', 'HSW',
                                          'HESAWA'),
                                          value ='HESAWA')

df['installer'] = df['installer'].replace(to_replace = ('Jaica', 'JAICA', 'Jica', 'Jeica', 'JAICA CO', 'JALCA',
                                          'Japan', 'JAPAN', 'JAPAN EMBASSY', 'Japan Government', 'Jicks',
                                          'JIKA', 'jika', 'jiks', 'Embasy of Japan in Tanzania', 'JICA'), 
                                          value ='JICA')

df['installer'] = df['installer'].replace(to_replace = ('KKT', 'KK', 'KKKT Church', 'KkKT', 'KKT C',
                                          'KKKT'), value ='KKKT')

df['installer'] = df['installer'].replace(to_replace = ('0', 'Not Known', 'not known', 'Not kno'), value ='Unknown')

In [115]:
len(df['installer'].value_counts())

1902

In [116]:
df['installer'].value_counts()[:35]

DWE                           17674
Central Government             3824
Community                      2189
DANIDA                         1767
HESAWA                         1499
RWE                            1220
District  Council              1184
KKKT                            950
Finnish Government              823
Unknown                         805
World Vision                    714
TCRS                            707
CES                             610
Ministry of Water               558
Roman Catholic Church           521
TASAF                           484
JICA                            457
LGA                             413
WEDECO                          400
NORAD                           396
DMDD                            376
World Bank                      354
UNICEF                          332
OXFAM                           331
AMREF                           329
TWESA                           316
WU                              301
ACRA                        

In [117]:
df['funder'].value_counts()[:60]

Government Of Tanzania            9084
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
Kkkt                              1287
World Vision                      1246
Unicef                            1057
Tasaf                              877
District Council                   843
Dhv                                829
Private Individual                 826
Dwsp                               811
0                                  777
Norad                              765
Germany Republi                    610
Tcrs                               602
Ministry Of Water                  590
Water                              583
Dwe                                484
Netherlands                        470
Hifab                              450
Adb                                448
Lga                                442
Amref                              425
Fini Water               

In [139]:
df['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [140]:
#reminder scheme_name has over half null values- will drop this column
df['scheme_name'].value_counts()

K                                  682
None                               644
Borehole                           546
Chalinze wate                      405
M                                  400
                                  ... 
Marine Park /Village                 1
BL Eligudi                           1
New keni Awater supply               1
Njalamatatawater gravity scheme      1
Mbawala chini water suplly           1
Name: scheme_name, Length: 2696, dtype: int64

In [141]:
#extraction columns are very similar, going to keep extraction_type_class as it is cleaner/concise (all "other.." are in one "other" column. 
df[['extraction_type_class', 'extraction_type_group' , 'extraction_type']].value_counts()

extraction_type_class  extraction_type_group  extraction_type          
gravity                gravity                gravity                      26780
handpump               nira/tanira            nira/tanira                   8154
other                  other                  other                         6430
submersible            submersible            submersible                   4764
handpump               swn 80                 swn 80                        3670
motorpump              mono                   mono                          2865
handpump               india mark ii          india mark ii                 2400
                       afridev                afridev                       1770
submersible            submersible            ksb                           1415
rope pump              rope pump              other - rope pump              451
handpump               other handpump         other - swn 81                 229
wind-powered           wind-powered  

In [142]:
#these have the same info just slightly different labels
df[['payment_type', 'payment']].value_counts()

payment_type  payment              
never pay     never pay                25348
per bucket    pay per bucket            8985
monthly       pay monthly               8300
unknown       unknown                   8157
on failure    pay when scheme fails     3914
annually      pay annually              3642
other         other                     1054
dtype: int64

In [143]:
#quality_group almost the same as water_quality, but water quality has slightly more details!
df[['water_quality', 'quality_group']].value_counts()

water_quality       quality_group
soft                good             50818
salty               salty             4856
unknown             unknown           1876
milky               milky              804
coloured            colored            490
salty abandoned     salty              339
fluoride            fluoride           200
fluoride abandoned  fluoride            17
dtype: int64

In [144]:
#quantity_group and quantity are duplicates
df[['quantity_group', 'quantity']].value_counts()

quantity_group  quantity    
enough          enough          33186
insufficient    insufficient    15129
dry             dry              6246
seasonal        seasonal         4050
unknown         unknown           789
dtype: int64

In [145]:
#source column has the most specific labels of the 3 similar source columns
df[['source_class', 'source_type', 'source']].value_counts()

source_class  source_type           source              
groundwater   spring                spring                  17021
              shallow well          shallow well            16824
              borehole              machine dbh             11075
surface       river/lake            river                    9612
              rainwater harvesting  rainwater harvesting     2295
groundwater   borehole              hand dtw                  874
surface       river/lake            lake                      765
              dam                   dam                       656
unknown       other                 other                     212
                                    unknown                    66
dtype: int64

In [149]:
#will One Hot Encode
df['public_meeting'].value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [150]:
#percent missing of public_meeting column:
pub_meet_null = (df['public_meeting'].isnull().sum() / len(df['public_meeting']))
pub_meet_null

0.05612794612794613

In [151]:
#will One Hot Encode
df['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [152]:
#percent missing of permit column:
perm_null = (df['permit'].isnull().sum() / len(df['permit']))
perm_null

0.05144781144781145

In [154]:
df.shape

(59400, 41)

In [155]:
#drop rows where permit and/or public_meeting have missing values:
df = df.dropna(subset=['permit', 'public_meeting'])

In [156]:
df.shape

(53281, 41)

In [157]:
53281/59400

0.896986531986532

As you can see, many categorical column pairs have duplicate and/or very similar information. The following are the categorical columns that I will drop as well as the reason to drop:\
**recorded_by**: This is the same string for every entry\
**management_group**: I will keep management which contains slightly more details of management\
**waterpoint_type_group**: redundant with waterpoint_type\
**payment_type**: redundant with payment\
**extraction_type** & **extraction_type_group**: keeping extraction_type_class so these are not necessary\
**date_recorded**: I'm keeping construction_year so this is not necessary\
**source_class** & **source_type**: dropping these as I'll be keeping source\
**quantity_group**: redundant with quantity\
**quality_group**: highly correlates with water_quality\
**wpt_name**: just a name for the waterpoint, won't help with modeling\
**subvillage**, **lga**, **ward**: can all be represented by region column\
**scheme_name**: contains too many unique values and too many nulls, keeping scheme_management\
**funder/installer????

In [158]:
df.drop(['recorded_by', 'management_group', 'waterpoint_type_group', 'payment_type', 'extraction_type', 'extraction_type_group', 
        'date_recorded', 'source_class', 'source_type', 'quantity_group', 'quality_group', 'wpt_name', 'subvillage', 'lga', 'ward',
         'scheme_name'], axis=1, inplace=True)

In [159]:
df.shape

(53281, 25)

### Cleaning Numerical Columns:

In [162]:
numerical = [var for var in df.columns if df[var].dtype!='O']
numerical

['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [163]:
#Take a look at Summary Statistics for Numerical Columns:
print("Numerical Feature Summary Statistics:")
numerical_stats = df.describe()
print(numerical_stats)

Numerical Feature Summary Statistics:
                 id     amount_tsh    gps_height     longitude      latitude  \
count  53281.000000   53281.000000  53281.000000  53281.000000  5.328100e+04   
mean   37092.441846     335.979816    659.404797     34.208161 -5.738222e+00   
std    21441.314241    2704.965314    692.691959      6.307645  2.895428e+00   
min        1.000000       0.000000    -90.000000      0.000000 -1.164944e+01   
25%    18514.000000       0.000000      0.000000     33.014412 -8.457237e+00   
50%    37038.000000       0.000000    352.000000     35.124693 -5.097481e+00   
75%    55610.000000      30.000000   1303.000000     37.270955 -3.337499e+00   
max    74247.000000  250000.000000   2770.000000     40.345193 -2.000000e-08   

        num_private   region_code  district_code    population  \
count  53281.000000  53281.000000   53281.000000  53281.000000   
mean       0.516563     15.295809       5.819542    178.880708   
std       12.877568     17.983823       9.9

In [161]:
#amount total static head- many records have 0.0 but technically, 0.0 could be accurrate, i.e. if the source is groundwater..
df['amount_tsh'].value_counts()[:35]

0.0        36428
500.0       3028
50.0        2095
1000.0      1440
20.0        1426
200.0       1193
10.0         800
2000.0       684
30.0         682
100.0        659
250.0        569
300.0        551
5000.0       444
5.0          375
3000.0       328
25.0         326
1200.0       266
1500.0       189
6.0          189
600.0        162
4000.0       153
2400.0       145
2500.0       137
6000.0       123
7.0           69
8000.0        60
750.0         59
40.0          58
10000.0       57
12000.0       51
20000.0       44
450.0         42
3600.0        42
400.0         42
2200.0        31
700.0         24
70.0          22
4700.0        22
150.0         20
33.0          20
15.0          15
60.0          15
15000.0       14
2800.0        14
2.0           13
7200.0        12
1300.0        10
25000.0        9
6500.0         9
7500.0         8
35.0           8
Name: amount_tsh, dtype: int64

In [41]:
from sklearn.ensemble import RandomForestClassifier

 # Assuming df is your DataFrame
X = df[['installer', 'funder']]
y = df['status_group']

# Convert categorical features to numerical using one-hot encoding if needed
X = pd.get_dummies(X, columns=['installer', 'funder'], drop_first=True) 

model = RandomForestClassifier(random_state=42)
model.fit(X, y)

feature_importances = pd.Series(model.feature_importances_, index=X.columns)
print(feature_importances.sort_values(ascending=False))

funder_Government Of Tanzania    2.871692e-02
installer_DWE                    2.265414e-02
installer_Government             2.092267e-02
installer_RWE                    2.070050e-02
installer_Central government     1.269599e-02
                                     ...     
installer_TCRS/ TASSAF           9.575295e-07
installer_Mara inter product     5.126127e-07
installer_TASSAF/TCRS            0.000000e+00
installer_kanisa                 0.000000e+00
installer_TASSAF/ TCRS           0.000000e+00
Length: 4040, dtype: float64


In [166]:
#Over 98% have value 0, will drop as this won't be helpful for modeling..
df['num_private'].value_counts()

0      52557
6         73
1         68
8         46
5         44
       ...  
755        1
180        1
213        1
23         1
94         1
Name: num_private, Length: 61, dtype: int64

The following are the numerical columns I will drop as well as the reason to drop:\
**id**: this is just a unique identifier, will not help with modeling\
**num_private**: over 98% have the value 0\
**region_code** & **district_code**: too similar/redundant with region column\

In [167]:
df.drop(['id', 'num_private', 'region_code', 'district_code'], axis=1, inplace=True)

In [168]:
df.shape

(53281, 21)

In [169]:
df.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,population,public_meeting,...,permit,construction_year,extraction_type_class,management,payment,water_quality,quantity,source,waterpoint_type,status_group
0,6000.0,Roman,1390,Roman Catholic Church,34.938093,-9.856322,Lake Nyasa,Iringa,109,True,...,False,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,functional
2,25.0,Lottery Club,686,World Vision,37.460664,-3.821329,Pangani,Manyara,250,True,...,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,functional
3,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,58,True,...,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,needs repair
4,0.0,Action In A,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,0,True,...,True,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,functional
5,20.0,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Pangani,Tanga,1,True,...,True,2009,submersible,vwc,pay per bucket,salty,enough,other,communal standpipe multiple,functional


In [170]:
df.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'basin', 'region', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year',
       'extraction_type_class', 'management', 'payment', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'status_group'],
      dtype='object')

## Modeling

# Conclusions

## Limitations

## Recommendations

## Next Steps